In [ ]:
!wget http://www.bioinf.jku.at/software/DeepSynergy/data_test_fold0_tanh.p.gz

--2021-07-08 14:42:47--  http://www.bioinf.jku.at/software/DeepSynergy/data_test_fold0_tanh.p.gz
Resolving www.bioinf.jku.at (www.bioinf.jku.at)... 140.78.90.4
Connecting to www.bioinf.jku.at (www.bioinf.jku.at)|140.78.90.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3996502965 (3.7G) [application/x-gzip]
Saving to: ‘data_test_fold0_tanh.p.gz’

data_test_fold0_tan 100%[===================>]   3.72G  35.8MB/s    in 70s     

2021-07-08 14:43:57 (54.2 MB/s) - ‘data_test_fold0_tanh.p.gz’ saved [3996502965/3996502965]



In [ ]:
import tensorflow as tf
import gzip
import pickle
import os
tf.random.set_seed(334)

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
file = gzip.open('data_test_fold0_tanh.p.gz', 'rb')

X_tr, X_val, X_train, X_test, y_tr, y_val, y_train, y_test = pickle.load(file)

del X_tr
del y_tr

In [ ]:
#creating the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(8846, )),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(1024, activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1)
])


#compiling the model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mse'],
              run_eagerly=True)

In [ ]:
count=0
best, _ = model.evaluate(X_test, y_test)
while 1:
  model.fit(X_train, y_train, epochs=5)
  count += 1
  print("test result:")
  loss, _ = model.evaluate(X_test, y_test)
  if loss < best:
    print('best model updated! loss it now', loss)
    model.save('/gdrive/My Drive/best_model.h5')
    best = loss
  print("epoch count:",count)
  if count%5 == 0:
    key=input("Do you want continue? y/n or(every thing)")
    if key!='y':
      break

285/285 [==============================] - 4s 4ms/step - loss: 285.6067 - mse: 285.6067
Epoch 1/10
1157/1157 [==============================] - 28s 23ms/step - loss: 278.8878 - mse: 278.8878
Epoch 2/10
1157/1157 [==============================] - 27s 24ms/step - loss: 255.3741 - mse: 255.3741
Epoch 3/10
1157/1157 [==============================] - 27s 24ms/step - loss: 234.3993 - mse: 234.3993
Epoch 4/10
1157/1157 [==============================] - 27s 23ms/step - loss: 227.2825 - mse: 227.2825
Epoch 5/10
1157/1157 [==============================] - 27s 24ms/step - loss: 204.8289 - mse: 204.8289
Epoch 6/10
1157/1157 [==============================] - 27s 23ms/step - loss: 188.4697 - mse: 188.4697
Epoch 7/10
1157/1157 [==============================] - 27s 24ms/step - loss: 184.4144 - mse: 184.4144
Epoch 8/10
1157/1157 [==============================] - 28s 24ms/step - loss: 175.5204 - mse: 175.5204
Epoch 9/10
1157/1157 [==============================] - 28s 24ms/step - loss: 164.7840 -

In [ ]:
model.fit(X_train, y_train, epochs=95, validation_data = (X_val, y_val))


Epoch 1/95
1157/1157 [==============================] - 42s 34ms/step - loss: 431.1185 - mse: 431.1185 - val_loss: 342.2583 - val_mse: 342.2583
Epoch 2/95
1157/1157 [==============================] - 41s 35ms/step - loss: 373.9376 - mse: 373.9376 - val_loss: 346.7945 - val_mse: 346.7945
Epoch 3/95
1157/1157 [==============================] - 41s 36ms/step - loss: 337.3473 - mse: 337.3473 - val_loss: 260.4380 - val_mse: 260.4380
Epoch 4/95
1157/1157 [==============================] - 38s 33ms/step - loss: 300.8743 - mse: 300.8743 - val_loss: 225.5896 - val_mse: 225.5896
Epoch 5/95
1157/1157 [==============================] - 38s 33ms/step - loss: 278.3470 - mse: 278.3470 - val_loss: 223.6873 - val_mse: 223.6873
Epoch 6/95
1157/1157 [==============================] - 41s 35ms/step - loss: 257.5017 - mse: 257.5017 - val_loss: 206.4746 - val_mse: 206.4746
Epoch 7/95
1157/1157 [==============================] - 38s 33ms/step - loss: 235.8334 - mse: 235.8334 - val_loss: 242.7530 - val_mse: 2

In [ ]:
model.evaluate(X_test, y_test)

285/285 [==============================] - 4s 4ms/step - loss: 229.8353 - mse: 229.8353


[229.83531188964844, 229.83531188964844]

In [ ]:
model.save('/gdrive/My Drive/best_model.h5')

In [ ]:
#load model
from tensorflow.keras.models import load_model

model = load_model('/gdrive/My Drive/best_model.h5')

In [ ]:

y_pred= model.predict(X_test)

from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('RMSE:', rmse)

RMSE: 15.160319842643142


In [ ]:
from scipy.stats import pearsonr
y_n=np.reshape(y_pred,(9108,))
pearson_r = pearsonr(y_n,y_test)
print("Pearson\'s r:", pearson_r)

Pearson's r: (0.6856798816419686, 0.0)


In [ ]:
y_n=np.reshape(y_pred,(9108,))
y_n[0:5]



array([-3.2676618 ,  0.31541526,  3.6097574 , -2.1480434 , -6.6353416 ],
      dtype=float32)